In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '..')

from datetime import datetime, timedelta
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit_ibm_runtime import QiskitRuntimeService

# Project imports
from src.calibration import CalibrationCollector, DriftAnalyzer, collect_daily_snapshot
from src.utils import QPUBudgetTracker, DataManager, setup_data_directories

## 1. Initialize IBM Quantum Service

Connect to IBM Quantum using your credentials.

In [ ]:
# Initialize service (assumes credentials are saved)
# If first time, run: QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_TOKEN")

try:
    service = QiskitRuntimeService()
    print("✅ Connected to IBM Quantum")
    print(f"Available backends: {len(service.backends())}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nTo set up credentials:")
    print('QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN")')

## 2. Select Target Backend

Choose a backend based on:
- Availability in your region (Open Plan constraint)
- Number of qubits (≥127 for full experiments)
- Dynamic circuit support (for mid-circuit measurement)

In [ ]:
# List available backends with properties
backends = service.backends()

print("Available backends:")
print("-" * 60)

for backend in backends:
    config = backend.configuration()
    num_qubits = config.num_qubits if hasattr(config, 'num_qubits') else 'N/A'
    print(f"{backend.name}: {num_qubits} qubits")

In [ ]:
# Select primary backend
BACKEND_NAME = "ibm_sherbrooke"  # 127-qubit Eagle processor

try:
    backend = service.backend(BACKEND_NAME)
    print(f"✅ Selected backend: {backend.name}")
    
    config = backend.configuration()
    print(f"   Qubits: {config.num_qubits}")
    print(f"   Basis gates: {config.basis_gates}")
except Exception as e:
    print(f"❌ Backend not available: {e}")
    print("Try another backend from the list above.")

## 3. Setup Data Directories

In [ ]:
# Initialize data management
dirs = setup_data_directories("../data")

print("Data directories created:")
for name, path in dirs.items():
    print(f"  {name}: {path}")

# Initialize budget tracker
budget_tracker = QPUBudgetTracker(tracking_file="../data/qpu_budget.json")
print(f"\n📊 QPU Budget remaining: {budget_tracker.get_remaining_budget():.0f}s ({budget_tracker.get_remaining_budget()/60:.1f} min)")

## 4. Collect Calibration Snapshot

This uses only the free backend properties API - no QPU time consumed.

In [ ]:
# Initialize calibration collector
collector = CalibrationCollector(data_dir="../data/calibration")

# Collect snapshot
print(f"Collecting calibration snapshot from {BACKEND_NAME}...")
snapshot_path = collect_daily_snapshot(service, BACKEND_NAME, collector)

print(f"✅ Snapshot saved to: {snapshot_path}")

In [ ]:
# Load and inspect snapshot
snapshots = collector.load_snapshots(BACKEND_NAME)
latest = snapshots[-1] if snapshots else None

if latest:
    print(f"Latest snapshot: {latest['timestamp']}")
    print(f"Calibration time: {latest['calibration_time']}")
    print(f"Number of qubits: {latest['num_qubits']}")
    print(f"Coupling map edges: {len(latest['coupling_map'])}")
    
    # Show sample qubit properties
    print("\nSample qubit properties (qubit 0):")
    q0 = latest['qubits'].get('0', {})
    for prop, data in q0.items():
        print(f"  {prop}: {data['value']:.6e} {data['unit']}")

## 5. Initial Drift Analysis

Analyze available calibration data for drift patterns.

In [ ]:
# Initialize drift analyzer
analyzer = DriftAnalyzer()

# Generate drift report
if len(snapshots) >= 2:
    report = analyzer.generate_drift_report(snapshots, BACKEND_NAME)
    
    print("Drift Analysis Report")
    print("=" * 40)
    print(f"Snapshots analyzed: {report['num_snapshots']}")
    print(f"Date range: {report['date_range']['start']} to {report['date_range']['end']}")
    print("\nRecommendations:")
    for rec in report['recommendations']:
        print(f"  • {rec}")
else:
    print("Need at least 2 snapshots for drift analysis.")
    print("Run this notebook daily to build calibration history.")

## 6. Visualize Qubit Quality

Create a heatmap of T1/T2 values across the backend.

In [ ]:
if latest:
    # Extract T1 values
    t1_values = []
    t2_values = []
    ro_errors = []
    
    for q_idx in range(latest['num_qubits']):
        q_data = latest['qubits'].get(str(q_idx), {})
        t1 = q_data.get('T1', {}).get('value', 0)
        t2 = q_data.get('T2', {}).get('value', 0)
        ro = q_data.get('readout_error', {}).get('value', 1)
        
        t1_values.append(t1 * 1e6 if t1 else 0)  # Convert to μs
        t2_values.append(t2 * 1e6 if t2 else 0)
        ro_errors.append(ro)
    
    # Plot distributions
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].hist(t1_values, bins=30, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('T1 (μs)')
    axes[0].set_ylabel('Count')
    axes[0].set_title(f'T1 Distribution ({BACKEND_NAME})')
    
    axes[1].hist(t2_values, bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[1].set_xlabel('T2 (μs)')
    axes[1].set_ylabel('Count')
    axes[1].set_title(f'T2 Distribution ({BACKEND_NAME})')
    
    axes[2].hist(ro_errors, bins=30, edgecolor='black', alpha=0.7, color='green')
    axes[2].set_xlabel('Readout Error')
    axes[2].set_ylabel('Count')
    axes[2].set_title(f'Readout Error Distribution ({BACKEND_NAME})')
    
    plt.tight_layout()
    plt.savefig('../figures/phase0_qubit_quality.pdf', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print summary statistics
    print(f"\nSummary Statistics:")
    print(f"T1: mean={np.mean(t1_values):.1f}μs, std={np.std(t1_values):.1f}μs")
    print(f"T2: mean={np.mean(t2_values):.1f}μs, std={np.std(t2_values):.1f}μs")
    print(f"Readout Error: mean={np.mean(ro_errors):.4f}, std={np.std(ro_errors):.4f}")

## 7. Identify Best Qubits

Rank qubits by quality for experiment planning.

In [ ]:
if latest:
    # Score each qubit
    qubit_scores = []
    
    for q_idx in range(latest['num_qubits']):
        q_data = latest['qubits'].get(str(q_idx), {})
        
        t1 = q_data.get('T1', {}).get('value', 0) or 0
        t2 = q_data.get('T2', {}).get('value', 0) or 0
        ro = q_data.get('readout_error', {}).get('value', 1) or 1
        
        # Simple scoring: higher T1/T2, lower readout error
        score = (t1 * 1e6 / 100) + (t2 * 1e6 / 100) + (1 - ro) * 10
        qubit_scores.append((q_idx, score, t1*1e6, t2*1e6, ro))
    
    # Sort by score
    qubit_scores.sort(key=lambda x: x[1], reverse=True)
    
    print("Top 20 Qubits by Quality Score:")
    print("-" * 60)
    print(f"{'Qubit':>6} {'Score':>8} {'T1(μs)':>10} {'T2(μs)':>10} {'RO Error':>10}")
    print("-" * 60)
    
    for q_idx, score, t1, t2, ro in qubit_scores[:20]:
        print(f"{q_idx:>6} {score:>8.2f} {t1:>10.1f} {t2:>10.1f} {ro:>10.4f}")

## Summary & Next Steps

✅ **Completed:**
- Connected to IBM Quantum
- Selected target backend
- Set up data directories
- Collected initial calibration snapshot
- Identified high-quality qubits

📋 **Next Steps:**
1. **Run daily:** Execute cells 4-5 daily to build calibration history
2. **Phase 1:** Once 7+ snapshots collected, proceed to probe suite validation
3. **Budget check:** Current remaining: shown above

⚠️ **Important:** This phase uses NO QPU time - only free calibration API.

In [ ]:
# Save phase completion status
phase_status = {
    "phase": 0,
    "completed_at": datetime.now().isoformat(),
    "backend": BACKEND_NAME,
    "snapshots_collected": len(snapshots),
    "qpu_time_used": 0,
    "ready_for_phase_1": len(snapshots) >= 7
}

with open('../data/phase_status.json', 'w') as f:
    json.dump(phase_status, f, indent=2)
    
print("Phase 0 status saved.")
print(f"Ready for Phase 1: {phase_status['ready_for_phase_1']}")